# Bienvenue sur le fichier de mise en forme

Avant de lancer ce script :
1. Placez le fichier excel des journaux provisoires que vous souhaitez formater dans le même dossier que ce notebook.
2. Assurez-vous que ce fichier s'appel "journal_provisoire_par_piece.xls"
3. Fermez le fichier excel "journal_provisoire_par_piece_clean.xls" s'il est ouvert autrement cela empêcherait l'export d'un nouveau fichier excel correctement formatté.

Après vérification, vous pouvez lancer le script :
1. Dans la barre au dessus, cliquez sur le bouton avec <button class="btn btn-default" title="redémarrer le noyau, et ré-exécuter tout le notebook (sans confirmation)"><i class="fa-forward fa"></i></button> les deux triangles noirs orientés à droites pour relancer et exécuter toutes les cellules.
2. Les cellules de ce notebook seront jouées les unes après les autres.
3. La toute dernière cellule produira le fichier excel correctement formatté, appelé "journal_provisoire_par_piece_clean.xls"

<i class='fa fa-warning'></i> S'il apparaît une fenêtre indiquant que le noyau a planté, alors le fichier excel "journal_provisoire_par_piece.xls" est certainement manquant.

In [264]:
# Importation de librairies
import sys
sys.tracebacklimit = 0
import re
import glob
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta, date
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import time
from pathlib import Path
from openpyxl import load_workbook

In [265]:
# On récupère le fichier dans le répertoire courant.

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
li = []
error_log=[]
filenames=[]
filename=glob.glob("journal_provisoire_par_piece.xls")[0]    
print(filename)
df=pd.read_excel(filename, engine="xlrd", header = None)
print("\nNe pas tenir compte du message ci-dessus ci-dessus.")

13-06-2022-11-07-26
journal_provisoire_par_piece.xls
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'

Ne pas tenir compte du message ci-dessus ci-dessus.


In [268]:
col_to_drop=[]
df_height,_=df.shape

print(df.head())
# On élimine les colonnes qui ne servent à rien
for col in df.columns:
    count = df[col].isna().sum()
    if (count/df_height)>0.95:
        col_to_drop.append(col)

col_to_drop.extend([10,12,13])
print(col_to_drop)


df_drop=df.drop(col_to_drop, axis=1)
df_drop = df_drop.iloc[3: , :]

print(df_drop.head(10))

           0              1     2                      3           4       5   \
0         NaN            NaN   NaN                    NaN         NaN     NaN   
1         NaN              1   NaN  SURFACE VOLUME ESPACE         NaN     NaN   
2         NaN            NaN  Date                    NaN         NaN  Compte   
3         NaN  Pièce : 20102   NaN                    NaN         NaN     NaN   
4  24/01/2022            NaN   NaN                    NaN  41100675.0     NaN   

         6                                     7        8    9   ...       13  \
0       NaN  Journal général provisoire par pièce      NaN  NaN  ...      NaN   
1       NaN                                   NaN      NaN  NaN  ...      NaN   
2       NaN                                   NaN  Ctr-Par  Jal  ...  Libellé   
3       NaN                                   NaN      NaN  NaN  ...      NaN   
4  512675.0                                   NaN      NaN   VI  ...      NaN   

    14          15        

In [269]:
df_alias=""
df_alias=df_drop


# Dans les colonnes 1 (Pièce) et 14 (Sous-Total), s'il y a une cellule vide, je la renseine à partir de la prochaine valeur non-vide situé en dessous d'elle.
# df_alias[1].ffill(inplace=True)
# df_alias[14].bfill(inplace=True)
df_alias[1].ffill(inplace=True)

# On créer un alias pour éviter de perdre la donnée déjà obtenue et travailler sur une copie
print("\n")

# On supprime toutes les lignes dont la colonne 0 (Date) est nulle
df_alias = df_alias.drop(df_alias[df_alias[0].isnull() & df_alias[14].isnull()].index)

# On remplace le mot "Pièce" par "" (rien) pour éviter que le tableau soit trop chargé inutillement
df_alias.replace("Pièce : ","",regex=True,inplace=True)
# df_alias.replace(np.nan,0,regex=True,inplace=True)
df_alias.replace(np.nan,"",regex=True,inplace=True)
df_alias[14].replace('Sous total pièce [0-9]+ :',"Sous-Total",regex=True,inplace=True)

df_alias.loc[df[0].isnull(),[1]] = ""

df_rename=df_alias

# On renomme les colonnes pour avoir des libellés clairs
df_rename.rename(columns={0: "Date"},inplace=True)
df_rename.rename(columns={1: "Pièce"},inplace=True)
df_rename.rename(columns={4: "Compte"},inplace=True)
df_rename.rename(columns={6: "Ctr-Par"},inplace=True)
df_rename.rename(columns={9: "Jal"},inplace=True)
df_rename.rename(columns={11: "Libellé"},inplace=True)
df_rename.rename(columns={12: "Solde"},inplace=True)
df_rename.rename(columns={14: ""},inplace=True)
df_rename.rename(columns={19: "Débit"},inplace=True)
df_rename.rename(columns={22: "Crédit"},inplace=True)


print(df_rename.head(10))



          Date  Pièce      Compte     Ctr-Par Jal  \
4   24/01/2022  20102  41100675.0    512675.0  VI   
5   24/01/2022  20102    512675.0  41100675.0  VI   
6                                                   
8   06/01/2022  22489  41100207.0    512001.0  VI   
9   06/01/2022  22489    512001.0  41100207.0  VI   
10                                                  
12  20/01/2022  22651  41100207.0    512001.0  VI   
13  20/01/2022  22651    512001.0  41100207.0  VI   
14                                                  
16  24/01/2022  22875  41100675.0    512675.0  VI   

                           Libellé               Débit  Crédit  
4   AGN: NETTOY PK SUITE INFILTRAT               497.7          
5   AGN: NETTOY PK SUITE INFILTRAT                       497.7  
6                                   Sous-Total   497.7   497.7  
8           MBS  TVX PEINTURE& SOL              1210.0          
9           MBS  TVX PEINTURE& SOL                      1210.0  
10                      

In [5]:
# result_df = df_alias.drop_duplicates(subset=['Libellé'], keep='first')
# sort_df=result_df.sort_values(by=['Libellé'])

In [263]:
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
# On exporte en un fichier excel propre
# Si erreur, vérifiez que le fichier excel n'est pas déjà ouvert
df_alias.to_excel('journal_provisoire_par_piece_clean.xlsx', engine='xlsxwriter',index=False)     
df_alias.to_excel(filename.split('.')[0]+'_clean_'+dt_string+'.xlsx', engine='xlsxwriter',index=False)     

print("Export réussi !\nLe fichier suivant a été créé :\n")
print(filename.split('.')[0]+'_clean_'+dt_string+'.xlsx')

Export réussi !
Le fichier suivant a été créé :

journal_provisoire_par_piece_clean_13-06-2022-11-07-02.xlsx
